In [13]:
import pandas_gbq
import os
from google.cloud import bigquery
from dotenv import load_dotenv
from google.cloud import storage
load_dotenv()

True

In [2]:
sql = """
SELECT * FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` LIMIT 100000
"""
df = pandas_gbq.read_gbq(sql, project_id=os.environ.get("PROJECT_ID"))

Downloading: 100%|██████████| 100000/100000 [00:38<00:00, 2626.09rows/s]


In [3]:
df_generated = df.copy()
df_generated.isna().any() # Checking which points are needed to fill in

unique_key                False
taxi_id                   False
trip_start_timestamp      False
trip_end_timestamp         True
trip_seconds               True
trip_miles                False
pickup_census_tract        True
dropoff_census_tract       True
pickup_community_area     False
dropoff_community_area     True
fare                       True
tips                       True
tolls                      True
extras                     True
trip_total                 True
payment_type              False
company                    True
pickup_latitude           False
pickup_longitude          False
pickup_location           False
dropoff_latitude           True
dropoff_longitude          True
dropoff_location           True
dtype: bool

In [5]:
df_sample = df.loc[~df[column].isnull()][column]

In [6]:
# Setting to 0 all Nans since Apply does not work with Nans
# We use the df instead of df_generated to generate random data points since df_generated has a probability of returning a 0 since the value is available there it can be improved using the .loc & instead
column = 'dropoff_latitude'
df_generated[column] = df_generated[column].fillna(0)
df_generated[column] =df_generated[column].apply(lambda x: df_sample.sample(1).item() if x == 0 else x)
column = 'dropoff_longitude'
df_generated[column] = df_generated[column].fillna(0)
df_generated[column] =df_generated[column].apply(lambda x: df_sample.sample(1).item() if x == 0 else x)

Lambda function to generate dropoff location is differen since it requires two columsn and a concat of strings. eg "POINT (-87.642648998 41.8792550844)"

In [7]:
df_generated['dropoff_location'] = df_generated['dropoff_location'].fillna(0)
df_generated['dropoff_location'] = df_generated.apply(lambda x: 'POINT ('+str(x['dropoff_longitude'])+' '+str(x['dropoff_latitude'])+ ')' if x['dropoff_location'] ==0 else x['dropoff_location'], axis=1)

In [8]:
df_generated['company1'] = df_generated['company'].fillna('CompanyNotAssigned')

Dropping off columns, selecting only those columns which are required

In [9]:
df_generated = df_generated[['unique_key','taxi_id','company','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','dropoff_location']]

Creating CSV file to upload

In [10]:
df_generated.to_csv('chicago-manual.csv',index=False)

Uploading CSV

In [21]:
bucket_name = "chicagobucket"
source_file_name = 'chicago-manual.csv'
destination_blob_name = 'manual/chicago-manual.csv'

storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)
print("File {} uploaded to {}.".format(source_file_name, destination_blob_name))

File chicago-manual.csv uploaded to manual/chicago-manual.csv.
